In [46]:
import cv2
import pandas as pd
import numpy as np
import json

In [47]:
with open('bboxes.json', 'r') as j:
     contents = json.loads(j.read())

In [67]:
def show_frames_w_bboxes(trim_name, df):
    vidcap = cv2.VideoCapture(f'Video/Локотех_{trim_name}.mp4')
    success, image = vidcap.read()
    count = 1
    
    while success:
        objects = df[df.frame == count]
        for _, obj in objects.iterrows():
            name = obj.label
        
            if name == 'bot':
                color = (0, 230, 255)
            elif name == 'mob':
                color = (188, 0, 255)
            else:
                color = (255, 255, 255)
                
            t = int(obj.top)
            h = int(obj.height)
            l = int(obj.left)
            w = int(obj.width)
                
            cv2.rectangle(image, (l + w, t), (l, t + h), color, 2)
            
        cv2.imshow('lol', image)
        cv2.waitKey(0)
        cv2.destroyWindow('lol')
        success, image = vidcap.read()
        count += 1

In [49]:
its = list(contents[2]['projects.clgwbo68i04k1071t3d8i4qo1.labels'][0]['annotations']['frames'].items())

In [50]:
matrix = []
for frame in its:
    frame_num = frame[0]
    objects = frame[1]['objects']
    obj_id = objects.keys()

    for aidi in obj_id:
        row = []
        bbox = [k[1] for k in [*objects[aidi]['bounding_box'].items()]]
        row.append([frame_num, aidi, objects[aidi]['name'], *bbox])
        matrix.extend(row)

In [51]:
df = pd.DataFrame(matrix, columns=['frame', 'object_id', 'label', 'top', 'left', 'height', 'width'])
df.frame = df.frame.astype(int)

In [52]:
pd.options.mode.chained_assignment = None 

monster_df = pd.DataFrame()
for obj in df.object_id.unique().tolist():
    over_df = pd.DataFrame()
    slice_bob = df[df.object_id == obj].sort_values('frame')
    for first, second in zip(slice_bob.frame.unique().tolist(), slice_bob.frame.unique().tolist()[1:]):
        frame_numbers = range(first, second + 1)
        sub_set_first = slice_bob[slice_bob.frame == first][['top', 'left', 'height', 'width']]
        sub_set_second = slice_bob[slice_bob.frame == second][['top', 'left', 'height', 'width']]
        temp_df = pd.DataFrame({'frame': frame_numbers, 
                                'object_id': obj, 
                                'label': slice_bob.label.iloc[0], 
                                'top': sub_set_first.top.iloc[0], 
                                'left': sub_set_first.left.iloc[0], 
                                'height': sub_set_first.height.iloc[0], 
                                'width': sub_set_first.width.iloc[0]})
        temp_df['top'].iloc[1:] = None
        temp_df['top'].iloc[-1] = sub_set_second.top.iloc[0]
        temp_df['left'].iloc[1:] = None
        temp_df['left'].iloc[-1] = sub_set_second.left.iloc[0]
        temp_df['height'].iloc[1:] = None
        temp_df['height'].iloc[-1] = sub_set_second.height.iloc[0]
        temp_df['width'].iloc[1:] = None
        temp_df['width'].iloc[-1] = sub_set_second.width.iloc[0]
        temp_df = temp_df.interpolate()
        over_df = pd.concat([over_df, temp_df])
    monster_df = pd.concat([monster_df, over_df])
    
monster_df = monster_df.drop_duplicates().sort_values('frame')

In [68]:
show_frames_w_bboxes('Trim3', monster_df)

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:1261: error: (-27:Null pointer) NULL window: 'lol' in function 'cvDestroyWindow'


In [54]:
def convert_to_yolo_v7(df):
    tmp = df.copy()
    tmp['top'] = tmp['top'] / 608
    tmp['left'] = tmp['left'] / 608
    tmp['height'] = tmp['height'] / 608
    tmp['width'] = tmp['width'] / 608
    tmp['label'] = tmp['label'].replace({'bot': 0, 'mob': 1})
    
    # tmp['rank'] = tmp.groupby('frame').object_id.rank(method='dense').astype(int)
    # tmp['filename'] = tmp['frame'].astype(str) + '_' + tmp['rank'].astype(str)
    return tmp.drop(columns=['object_id']).reset_index(drop=True)

In [55]:
df_yolo = convert_to_yolo_v7(monster_df)

In [56]:
monster_df

,frame,object_id,label,top,left,height,width
0,1,clgwczv1p001a356jsdzjjxll,bot,528.0,331.0,191.0,133.0
0,1,clgwde9rc001s356joqdedjv9,mob,183.0,640.0,60.0,22.0
0,1,clgwde66w001o356jka0sdgs6,mob,191.0,600.0,92.0,33.0
0,1,clgwd3nhy001k356j4eixwwax,bot,168.0,573.0,67.0,23.0
1,2,clgwczv1p001a356jsdzjjxll,bot,522.4,337.6,196.6,131.2
...,...,...,...,...,...,...,...
56,2637,clgwde9rc001s356joqdedjv9,mob,171.0,632.0,61.0,29.0
57,2638,clgwde9rc001s356joqdedjv9,mob,171.0,632.0,61.0,29.0
58,2639,clgwde9rc001s356joqdedjv9,mob,171.0,632.0,61.0,29.0
59,2640,clgwde9rc001s356joqdedjv9,mob,171.0,632.0,61.0,29.0


In [57]:
df_yolo

,frame,label,top,left,height,width
0,1,0,0.868421,0.544408,0.314145,0.218750
1,1,1,0.300987,1.052632,0.098684,0.036184
2,1,1,0.314145,0.986842,0.151316,0.054276
3,1,0,0.276316,0.942434,0.110197,0.037829
4,2,0,0.859211,0.555263,0.323355,0.215789
...,...,...,...,...,...,...
5616,2637,1,0.281250,1.039474,0.100329,0.047697
5617,2638,1,0.281250,1.039474,0.100329,0.047697
5618,2639,1,0.281250,1.039474,0.100329,0.047697
5619,2640,1,0.281250,1.039474,0.100329,0.047697


In [115]:
import pybboxes as pbx

voc_bbox = (331, 528, 133, 191)
W, H = 608, 608  # WxH of the image
ff = pbx.convert_bbox(voc_bbox, from_type="coco", to_type="fiftyone", image_size=(W, H))
pbx.convert_bbox(ff, from_type="fiftyone", to_type="yolo", image_size=(W, H))

(0.6537828947368421, 1.0254934210526316, 0.21875, 0.31414473684210525)

In [116]:
# do not run anything below this line

In [11]:
def save_frames(trim_name, df):
    
    vidcap = cv2.VideoCapture(f'Video/Локотех_{trim_name}.mp4')
    success, image = vidcap.read()
    count = 1
    
    while success:
        image = cv2.resize(image, (608, 608))
        cv2.imwrite(f"Images/{trim_name[-1]}_{count}.jpg", image)
        annotations = df[df.frame == count].drop(columns='frame')
        annotations.to_csv(f"Images/{trim_name[-1]}_{count}.txt", header=None, index=None, sep=' ')
        
        success,image = vidcap.read()
        count += 1

In [12]:
save_frames('Trim3', df_yolo)

In [13]:
import glob
import os
import random 

def train_val_split(folder):
    if os.getcwd() != folder:
        os.chdir(f"{folder}")
    
    images = glob.glob("*.jpg")
    annots = glob.glob("*.txt")
    
    val_images = random.sample(images, int(len(images) * 0.2))
    test_images = random.sample(val_images, int(len(val_images) * 0.3))
    
    train_folder = '/Users/dushamishkin/Desktop/Jupyter/HAR/yolov7/train'
    test_folder = '/Users/dushamishkin/Desktop/Jupyter/HAR/yolov7/test'
    val_folder = '/Users/dushamishkin/Desktop/Jupyter/HAR/yolov7/valid'
    
    for name in images:
        name = name[:-4]
        if f"{name}.jpg" in val_images:
            
            if f"{name}.jpg" in test_images:
                os.replace(f"{folder}/{name}.jpg", f"{test_folder}/images/{name}.jpg")
                os.replace(f"{folder}/{name}.txt", f"{test_folder}/labels/{name}.txt")
            
            else:
                os.replace(f"{folder}/{name}.jpg", f"{val_folder}/images/{name}.jpg")
                os.replace(f"{folder}/{name}.txt", f"{val_folder}/labels/{name}.txt")
            
        else:
            os.replace(f"{folder}/{name}.jpg", f"{train_folder}/images/{name}.jpg")
            os.replace(f"{folder}/{name}.txt", f"{train_folder}/labels/{name}.txt")
        
    os.chdir('/Users/dushamishkin/Desktop/Jupyter/HAR/')

In [14]:
train_val_split('/Users/dushamishkin/Desktop/Jupyter/HAR/Images')